Hi,

I found the setup of pystan a little tricky. It needs a compiler. If you haven't set it up yet, I think you can find this link useful.

https://pystan2.readthedocs.io/en/latest/windows.html

In this tutorial, the difficulty is finding distutils.cfg. If your setting is default, I think you will find it in \anaconda3\Lib\distutils.


The following is our model and data that I have initially sorted out based on the professor's documents. According to the professor's prompt, I think we need the following three tasks：

    1. Compare the ability of your model to predict the winner of new (unseen) games to simple approaches, such as fraction of games won, number of games played, etc.
    
    2. Try evaluating how many games are required to accurately predict the players' skill levels / win probability by decreasing the amount of training data available and observing the performance. 
    
    4. Experiment with learning a more complex model, for example taking into account game features (player's selected character) or additional latent scores (such as offensive and defensive skill) along with a correspondingly more elaborate probability of win function.

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [3]:
import pickle
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model.pkl', 'rb'))
# except:  # ow, compile and save compiled model
sm = pystan.StanModel(model_code = skill_model)
with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

In [49]:
def load_data(path):
    with open(path, encoding='utf-8') as f: lines = f.read().split('\n')

    p = 0;playerid = {};
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue;   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid: playerid[player0]=p;p+=1
        if player1 not in playerid: playerid[player1]=p;p+=1

    nplayers = len(playerid)
    games = 0
    
    # Sparsifying parameters (discard some training examples):
    pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) );
    PA, PB, win = [], [], []
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue;   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1; nplays[b,a]+=1;nwins[a,b] += aw; nwins[b,a] += bw; PA.append(a+1); PB.append(b+1);games+=1
                if aw:
                    win.append(1)
                else:
                    win.append(0)


    return nplayers, games, PA, PB, win, nplays, nwins

In [50]:
nplayers, games, PA, PB, win, tnplays, tnwins = load_data('train.csv')

In [51]:
print('number of players', nplayers)
print('number of games', games)
print('number of PA wins vs PB', len(win))
print('number of PA', len(PA))
print('number of PB', len(PB))

number of players 999
number of games 4677
number of PA wins vs PB 4677
number of PA 4677
number of PB 4677


In [26]:
skill_data = {
    'N': nplayers,
    'E': games,
    'scale': 0.3,
    'win':win,
    'PA': PA,
    'PB': PB
}

In [27]:
fit = sm.sampling(data=skill_data, iter=1000, chains=4)

In [28]:
samples = fit.extract()

In [43]:
def logit(z): return 1./(1.+np.exp(-z))

# randomly choose two players

prob = logit( skill_data['scale']*(samples['skill'][:,996]-samples['skill'][:,997]) ).mean()

print(prob)

0.344822227002474


In [52]:
nplayers, games, PA, PB, win, vnplays, vnwins = load_data('valid.csv')

In [53]:
print('number of players', nplayers)
print('number of games', games)

number of players 999
number of games 4768


In [36]:
vnplays = vnplays.astype(np.int64)
vnwins = vnwins.astype(np.int64)

bi_loss = 0.
for i in range(nplayers):
    for j in range(i+1, nplayers):
        if vnplays[i, j] != 0:
            
            real_prob = vnwins[i,j] / vnplays[i,j] #real win rate
            predicted_prob = logit( skill_data['scale']*(samples['skill'][:,i]-samples['skill'][:,j]) ).mean() #predicted win rate
        
            bi_loss += np.logical_xor(real_prob > 0.5, predicted_prob > 0.5) 
            


total = (vnplays > 0).sum()
bi_loss /= total

In [37]:
bi_loss

0.1975266231535555